## Detect watermark 

*Note*: This is only for white semitransparent watermark. 

Find the pixels that contain a transparency 

In [1]:
import cv2 as cv
import numpy as np 

In [2]:
image = "./images/test1.jpg"
# image = "./images/test2-nored.jpg"
# image = "./images/test3.jpg"

In [3]:
watermarked = cv.imread(image)
# display
def display(name, img):
    cv.startWindowThread()
    cv.imshow(name, img/255)
    cv.waitKey(0)
    cv.destroyWindow(name)
    cv.waitKey(1)

display("Watermarked", watermarked)


In [4]:
# obtain the alpha mask

# rectangle to identify watermark
# test1  
x, y, w, h = 600, 1400, 250, 35 
# test2 
# x, y, w, h = 580, 1480, 470, 70 
# test3 
# x, y, w, h = 600, 1400, 250, 35 

selected = np.zeros_like(watermarked)
cv.rectangle(selected, (x,y), (x+w, y+h), (255,255,255), -1)

# greyscale (for darker images such as test1)
watermark = cv.bitwise_and(watermarked, selected) 
watermark = watermark.astype(float)
display("Selected", watermark)
grey = watermark.max(axis=-1)
grey[grey>200] = 255 
grey[grey!=255] = 0 
display("Alpha mask", grey)

# for lighter images (test2 and test3)
# highlighted = cv.imread("./images/test2.jpg")
# watermark = cv.bitwise_and(highlighted, selected) 
# watermark = watermark.astype(float)
# display("Selected", watermark)
# watermark[np.where((watermark == [0,0,255]).all(axis = 2))] = [255,255,255]
# watermark[np.where((watermark != [0,0,255]).all(axis = 2))] = [0,0,0]
# grey = watermark.max(axis=-1)
# grey[grey>0] = 255
# display("Mask", grey)

# HSV (alternative)
# hsv = cv.cvtColor(watermark, cv.COLOR_BGR2HSV)
# sensitivity = 50
# lower_white = np.array([0, 0, 255-sensitivity])
# upper_white = np.array([255, sensitivity, 255])
# hsv_mask = cv.inRange(hsv, lower_white, upper_white)
# display("Mask", hsv_mask)


In [5]:
W = grey
alpha_0 = 1.1* watermark[W!=0].mean()/255 # estimate alpha 
alpha = np.zeros_like(W)
alpha[W==255] = alpha_0
print("Alpha is ", alpha_0)

# for light images 
# W = grey 
# alpha = np.zeros_like(W)
# alpha[W>0] = 1 

alpha = np.repeat(alpha, 3, axis=-1).reshape(*(alpha.shape),3)
W = np.repeat(W, 3, axis=-1).reshape(*(W.shape),3)

Alpha is  0.9874781183456597


In [6]:
# NOT USED 
def blur_mask(watermarked, alpha, W):
    I = (watermarked.astype(float) - alpha * W)/(1-alpha)
    I[I<0] = 0 
    I[I>255] = 255 
    I = I.astype(uint8)

    fI = I.copy()
    fI[y:y+h, x:x+w, :] = cv.medianBlur(fI[y:y+h, x:x+w, :], 5) 

 

In [7]:
def process(watermarked, alpha, W, bg, output_name="removed.png"):
    aW = (alpha * W)
    # a1 = (1 / (1-alpha))
    med = bg 
    watermarked = watermarked.astype(float)
    J = watermarked

    # display("Watermarked", J)
    # display("aW", aW)
    I = (J - aW) # * a1, but for test1, alpha is around 1
    # display("I", I)
    I[I<0] = 0
    I[I>255] = 255 
    I = I.astype(np.uint8)

    display("Output", I) 

    # blur 
    fI = I.copy() 
    fI[y:y+h, x:x+w, :] = cv.medianBlur(fI[y:y+h, x:x+w, :], 7) 
    fI[fI<0] = 0
    fI[fI>255] = 255 

    display("Output2", fI)
    cv.imwrite(output_name, fI)

In [8]:
process(watermarked, alpha, W, W, "./images/removed1.png")